In [ ]:
# !pip uninstall tidyzoning -y
# !pip install git+https://github.com/HOUPU1993/tidyzoning.git

Found existing installation: tidyzoning 0.1.0
Uninstalling tidyzoning-0.1.0:
  Successfully uninstalled tidyzoning-0.1.0
  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-yrotl3ka
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit 1984dd75df539a1c71d109abddf6eea9f890ded1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tidyzoning: filename=tidyzoning-0.1.0-py3-none-any.whl size=3761 sha256=0e3a544f4c64af5316d501f8fc28fcf7b9b8f4966b4fd51085d9b70be6050895
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-0mckbmdk\wheels\f4\29\51\e431eb95e0bf6589310375ce483daada9c916a6f65f3b65a8f
Successfully built tidyzoning


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-yrotl3ka'


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json

from shapely.geometry import Point
from shapely.geometry import box

# input my libraries
from tidyzoning import find_district_idx
from tidyzoning import find_bldg_type
from tidyzoning import check_function

In [1]:
tidybuilding = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data/Tidybuildings/tidybuilding_2family.geojson")
tidyparcel = gpd.read_file(r"C:/Users/Admin/Desktop/zoning_data/Test_Data/Test_Parcels/testparcel_100per_correct.geojson")
tidyzoning = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data//Tidyzoning/tidyzoning_Haltom_City.geojson")

tidyzoning = tidyzoning.to_crs(tidyparcel.crs)
tidybuilding = tidybuilding.to_crs(tidyparcel.crs)

NameError: name 'gpd' is not defined

#### The Case Study to use find_district_idx function

In [81]:
find_district_idx_results = find_district_idx(tidyparcel, tidyzoning)
find_district_idx_results

,prop_id,object_id,zoning_id
3,03027805,1000,1.0
8,03027783,1001,1.0
10,03027791,1002,1.0
18,00684619,1003,3.0
22,06687849,1004,3.0
...,...,...,...
75858,03027848,995,1.0
75863,03027856,996,1.0
75868,03027821,997,1.0
75871,04723295,998,1.0


#### The Case Study to use find_bldg_type_results function

In [83]:
find_bldg_type_results = find_bldg_type(tidybuilding)
find_bldg_type_results

'2_family'

#### The Case Study to use check_land_use function

In [77]:
result_df = check_land_use(tidybuilding, tidyzoning)
result_df

,zoning_id,allowed
0,0,False
1,1,False
2,3,True
3,6,False
4,7,False
5,8,False
6,9,False
7,10,False
8,11,False
9,12,False
